# CCTS 40500: ML Midterm
### Abdallah Aboelela
File should be run one level outside the data folder

In [5]:
import pandas as pd 
import numpy as np
import os
import csv
import random

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder

# Questions A/B
#### Predict POS for Males and Females using infectious disease history

Here, we will attempt to predict POS using four models (Decision Tree, Random Forest, Naive Bayes, and Linear Regression, and based on the AUC and ROC curve create a model that randomly chooses between them.

In [6]:
# Contains cols sex, diagnosis, state, county, functions in appendix
infectious, le = encode(read_file('data/Infectious'))

In [7]:
#  Using label encoder values printed above to filter by gender
infectious_m = infectious[infectious.sex == le.index('M')]
infectious_f = infectious[infectious.sex == le.index('F')]

mX_train, mX_test, my_train, my_test = train_test_split(infectious_m.iloc[:, 1:].drop('diagnosis', axis = 1), 
                                                        infectious_m.diagnosis, random_state = 42)

fX_train, fX_test, fy_train, fy_test = train_test_split(infectious_f.iloc[:, 1:].drop('diagnosis', axis = 1), 
                                                        infectious_f.diagnosis, random_state = 42)

In [8]:
nb = GaussianNB()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators = 100)
reg = LinearRegression()

classifiers = [nb, dt, rf, reg]
names = ['NB', 'DT', 'RF', 'LinReg']

cols = ['clf','M', 'F']
rows = []

for i, clf in enumerate(classifiers):
    clf.fit(mX_train, my_train)
    y_pred = clf.predict(mX_test)
    m_auc = roc_auc_score(my_test, y_pred)
    
    clf.fit(fX_train, fy_train)
    y_pred = clf.predict(fX_test)
    f_auc = roc_auc_score(fy_test, y_pred)
    
    rows.append([names[i], m_auc, f_auc])

infectious_clfs = pd.DataFrame(rows, columns = cols).set_index('clf')

In [9]:
infectious_clfs

,M,F
clf,,
NB,0.582018,0.592048
DT,0.611511,0.676330
RF,0.591837,0.696429
LinReg,0.680163,0.713290


Across the board, we can see that it is easier to predict autism diagnosis for women than it is for men, and that a linear regression is particularly good, and crosses the 65% AUC boundry in both cases. Decision Tree and Random Forest also work well for women.

# Questions C/D/E
#### Predict POS for Males/Females using any/all combination of diseases using weeks up to 100, 150, and 200 and identify the most predictive diseases

Strategy: Run the four predictive methods separately by gender, disease, and number of weeks considered, to identify which are best. Then, we will combine the most useful diseases into one merged df on c_id, and create a combined classifier that chooses between the best classifiers.

In [10]:
weeks = [100, 150, 200]
cols = ['disease', 'clf', 'weeks', 'm_auc', 'f_auc']
names = ['NB', 'DT', 'RF', 'LinReg']
rows = []

# This takes a few minutes to run
# Try except blocks for when the code fails due to y_pred or y_true only have one value due to the train/test split
for fname in os.listdir('data'):
    if fname not in ['Positive', 'Negative'] and '.' not in fname and '_' not in fname:
        disease, le = encode(read_file('data/' + fname))
        disease_m = disease[disease.sex == le.index('M')]
        disease_f = disease[disease.sex == le.index('F')]
        
        mX_train, mX_test, my_train, my_test = train_test_split(disease_m.iloc[:, 1:].drop('diagnosis', axis = 1), 
                                                        disease_m.diagnosis, random_state = 42)

        fX_train, fX_test, fy_train, fy_test = train_test_split(disease_f.iloc[:, 1:].drop('diagnosis', axis = 1), 
                                                        disease_f.diagnosis, random_state = 42)
        
        for i, clf in enumerate(classifiers):
            for num_weeks in weeks:             
                clf.fit(mX_train.iloc[:, 1:num_weeks + 2], my_train)
                y_pred = clf.predict(mX_test.iloc[:, 1:num_weeks + 2])
                
                try:
                    m_auc = roc_auc_score(my_test, y_pred)
                except Exception as e:
                    m_auc = None

                clf.fit(fX_train.iloc[:, 1:num_weeks + 2], fy_train)
                y_pred = clf.predict(fX_test.iloc[:, 1:num_weeks + 2])
                
                try:
                    f_auc = roc_auc_score(fy_test, y_pred)
                except Exception as e:
                    f_auc = None
                                    
                new_row = [fname, names[i], num_weeks, m_auc, f_auc]
                print(new_row) # Hidden in submission - this is just to keep track as it runs
                rows.append(new_row)

['Musculoskeletal', 'NB', 100, 0.5944175760079714, 0.5140382317801673]
['Musculoskeletal', 'NB', 150, 0.5991080759229105, 0.5385304659498208]
['Musculoskeletal', 'NB', 200, 0.6205677206114663, 0.48805256869772995]
['Musculoskeletal', 'DT', 100, 0.5155904440177899, 0.5830346475507765]
['Musculoskeletal', 'DT', 150, 0.5576712761562204, 0.5806451612903226]
['Musculoskeletal', 'DT', 200, 0.579130920844776, 0.5824372759856631]
['Musculoskeletal', 'RF', 100, 0.5217391304347826, 0.5833333333333334]
['Musculoskeletal', 'RF', 150, 0.5217391304347826, 0.5833333333333334]
['Musculoskeletal', 'RF', 200, 0.5217391304347826, 0.5833333333333334]
['Musculoskeletal', 'LinReg', 100, 0.6311517243055387, 0.6814516129032258]
['Musculoskeletal', 'LinReg', 150, 0.5606484069312466, 0.5457487056949423]
['Musculoskeletal', 'LinReg', 200, 0.643072398959827, 0.5632218239745121]
['Development', 'NB', 100, 0.49324549447220967, 0.5809165771571786]
['Development', 'NB', 150, 0.47946388005452073, 0.6215538847117794]
[

['Cardiovascular', 'NB', 100, 0.5442488262910798, 0.4779796311588219]
['Cardiovascular', 'NB', 150, 0.44624413145539904, 0.6237269474263694]
['Cardiovascular', 'NB', 200, 0.5715962441314554, 0.6295072942471787]
['Cardiovascular', 'DT', 100, 0.5561032863849765, 0.49614643545279385]
['Cardiovascular', 'DT', 150, 0.5201291079812207, 0.49710982658959535]
['Cardiovascular', 'DT', 200, 0.5841549295774648, 0.49710982658959535]
['Cardiovascular', 'RF', 100, 0.5333333333333333, 0.5]
['Cardiovascular', 'RF', 150, 0.5306924882629108, 0.5]
['Cardiovascular', 'RF', 200, 0.5666666666666667, 0.5]
['Cardiovascular', 'LinReg', 100, 0.5021126760563381, 0.535645472061657]
['Cardiovascular', 'LinReg', 150, 0.45469483568075125, 0.5298651252408477]
['Cardiovascular', 'LinReg', 200, 0.49530516431924887, 0.3170933113129645]
['Infectious', 'NB', 100, 0.4740597702809611, 0.551306678049706]
['Infectious', 'NB', 150, 0.5185912228813031, 0.5858708025042686]
['Infectious', 'NB', 200, 0.5227183200769451, 0.583878770

In [11]:
disease_week_gender_clfs = pd.DataFrame(rows, columns = cols)
combos_m = disease_week_gender_clfs.drop('f_auc', axis = 1)
combos_f = disease_week_gender_clfs.drop('m_auc', axis = 1)

#### To identify the most predictive diseases, weeks considered, and to eventually decide what is the best combination of diseases to use, we look at the above dataframe and groupby various columns.

In [12]:
print(combos_m.groupby('weeks').mean().sort_values('m_auc'))
print(combos_m.groupby('weeks').max().sort_values('m_auc'))
print(combos_f.groupby('weeks').mean().sort_values('f_auc'))
print(combos_f.groupby('weeks').max().sort_values('f_auc'))

          m_auc
weeks          
150    0.543398
100    0.552606
200    0.568577
       disease clf     m_auc
weeks                       
150    Urinary  RF  0.780488
200    Urinary  RF  0.926829
100    Urinary  RF  1.000000
          f_auc
weeks          
100    0.564858
200    0.582004
150    0.590131
       disease clf     f_auc
weeks                       
200    Urinary  RF  0.780878
100    Urinary  RF  0.872210
150    Urinary  RF  0.889509


#### When grouping by weeks and gender, it appears that when considering male disease history, it is best to consider all 200 weeks - in terms of both mean and maximum AUC). For female disease history, it appears that it is better to use only the first 150 weeks.

#### Now we do the same but consider each disease separately, after dropping values that don't consider all 200 weeks for men or 150 weeks for women. This leaves us with 76 rows for each gender.

In [13]:
combos_m = combos_m[combos_m.weeks == 200]
combos_f = combos_f[combos_f.weeks == 150]

In [14]:
print(combos_m.groupby('disease').mean().sort_values('m_auc', ascending = False))
print(combos_m.groupby('disease').max().sort_values('m_auc', ascending = False))

                  weeks     m_auc
disease                          
Neoplastic          200  0.717236
Reproductive        200  0.622051
Respiratory         200  0.617111
Musculoskeletal     200  0.591128
Procedural          200  0.584927
Development         200  0.584700
Integumentary       200  0.584253
Hepatic             200  0.576220
Infectious          200  0.567722
Otic                200  0.563631
Ophthalmological    200  0.561807
Urinary             200  0.561438
Immune              200  0.558461
Cardiovascular      200  0.554431
PNS                 200  0.544639
Hematologic         200  0.528659
Digestive           200  0.527275
Metabolic           200  0.519193
Endocrine           200  0.438084
                 clf  weeks     m_auc
disease                              
Hepatic           RF    200  0.926829
Neoplastic        RF    200  0.750000
Respiratory       RF    200  0.732294
Reproductive      RF    200  0.656232
Musculoskeletal   RF    200  0.643072
Metabolic         RF

#### For men: Interestingly, Neoplastic, Reproductive, and Respiratory disease history appear in the top 5 for both mean and maximum AUCs when grouped by disease history. We will also consider Hepatic because of the very high max AUC. As such, we distill the df one more time to reduce the number of combinations considered manually. We do the same for women afterwards.

In [15]:
best_m_diseases = ['Neoplastic', 'Reproductive', 'Respiratory', 'Hepatic']
combos_m = combos_m[combos_m.disease.isin(best_m_diseases)]

In [16]:
print(combos_f.groupby('disease').mean().sort_values('f_auc', ascending = False))
print(combos_f.groupby('disease').max().sort_values('f_auc', ascending = False))

                  weeks     f_auc
disease                          
Reproductive        150  0.792039
Otic                150  0.663939
Development         150  0.629714
Infectious          150  0.629147
Metabolic           150  0.616915
Procedural          150  0.613741
Respiratory         150  0.598399
Immune              150  0.596376
PNS                 150  0.589025
Musculoskeletal     150  0.562064
Ophthalmological    150  0.551065
Cardiovascular      150  0.537675
Digestive           150  0.519646
Integumentary       150  0.492473
Neoplastic          150  0.459740
Endocrine           150       NaN
Hematologic         150       NaN
Hepatic             150       NaN
Urinary             150       NaN
                 clf  weeks     f_auc
disease                              
Reproductive      RF    150  0.889509
Respiratory       RF    150  0.759917
Otic              RF    150  0.733352
Procedural        RF    150  0.688275
Development       RF    150  0.683136
Metabolic         RF

#### For women, Reproductive, OTIC, and Development diseases seem to work well. We will consider these in our final models.

In [17]:
best_f_diseases = ['Reproductive', 'Otic', 'Development']
combos_f = combos_f[combos_f.disease.isin(best_f_diseases)]

#### For both men, and women, we drop the algorithm with the lowest mean AUC for the disease histories we are considering

In [18]:
print(combos_m.groupby('clf').mean().sort_values('m_auc'))
print(combos_f.groupby('clf').mean().sort_values('f_auc'))

        weeks     m_auc
clf                    
NB        200  0.564743
RF        200  0.605811
DT        200  0.620160
LinReg    200  0.741904
        weeks     f_auc
clf                    
NB        150  0.664493
DT        150  0.672017
RF        150  0.675747
LinReg    150  0.768666


#### In both cases, Naive Bayes does worse than the others when considering the AUCs below. Now, we create models for each of the genders.

In [19]:
print('Men: ', run_on_combined(combine(best_m_diseases, 'M')))
print('Women: ', run_on_combined(combine(best_f_diseases, 'F')))

Men:  0.5613614573346117
Women:  0.6822322869488261


#### Unfortunately, while these diseases worked fine on their own, they don't seem to be working very well when combined together (esp for Men). Through separate testing, I found that the below diseases seem to work better than expected. For men, using Neoplastic alone ended up doing a lot better than any combination of diseases that I tried 

In [20]:
# Men
run_on_combined(combine(['Neoplastic'], 'M'))

0.7465397923875433

In [21]:
# Women
run_on_combined(combine(['PNS', 'Urinary', 'Endocrine'], 'F'))

0.747994652406417

In [22]:
# Both using all four diseases + Reproductive and 150 weeks
run_on_combined(combine(['PNS', 'Urinary', 'Endocrine', 'Neoplastic', 'Reproductive']))
# Not very helpful

0.5783852290475765

# Question F
#### We run the same functions as above with new features set as 'True' (by default it is False). I chose the features sickness as a proportion of total time, 6 mos, 1 year, and two years - as well as a sum of the total time sick. However, this does not seem to be very helpful and only marginally imrpoves the results for women

In [23]:
print('Men: ', run_on_combined(combine(['Neoplastic'], gender = 'M', new_features = True)))
print('Women: ', run_on_combined(combine(['PNS', 'Urinary', 'Endocrine'], gender = 'F', new_features = True)))

Men:  0.7448096885813148
Women:  0.7486631016042781


In [24]:
run_on_combined(combine(['Neoplastic', 'PNS', 'Urinary', 'Endocrine', 'Reproductive'], new_features = True))

0.5745152734589355

# Appendix - Functions Used

In [25]:
'''
Takes a disease file, and reads into a pandas dataframe.
Creates from values: gender, diagnosis, state, and county
'''
def read_file(fname):
    with open(fname) as f:
        content = f.readlines()
    
    content = [x.strip().split(' ') for x in content]

    max_cols = 0
    for i, line in enumerate(content):
        max_cols = max(max_cols, len(line))

    name = fname[5:]

    columns = ['p_id', 'c_id'] + [name + str(i) for i in range(max_cols - 2)]

    df = pd.read_csv(fname, names = columns, engine = 'python', 
        delim_whitespace = True, index_col = 'c_id')

    df['sex'] = df.p_id.apply(lambda x: x[0])
    df['diagnosis'] = df.p_id.apply(lambda x: x[1:4])
    df['state'] = df.p_id.apply(lambda x: x[4:6])
    df['county'] = df.p_id.apply(lambda x: x[6:9])
    
    return df

In [26]:
def encode(df):
    df = df.replace(np.nan, -1)
    df = df.fillna(value = -1)

    for col in df.columns[1:]:
        le = LabelEncoder()
        le = le.fit(df[col].astype(str))
        
        df[col] = le.transform(df[col].astype(str))
        
        if col == 'sex':
            final_le = list(le.classes_)
            
    return df, final_le

In [27]:
def run_on_combined(combined):
    dt = DecisionTreeClassifier()
    rf = RandomForestClassifier(n_estimators = 100)
    rg = LinearRegression()
    
    X_train, X_test, y_train, y_test = train_test_split(combined.drop(['diagnosis', 'p_id'], axis = 1), 
                                                        combined.diagnosis, random_state = 42)
        
    dt.fit(X_train, y_train)
    rf.fit(X_train, y_train)
    rg.fit(X_train, y_train)
    
    dt_pred = pd.DataFrame({'dt' : dt.predict(X_test)})
    rf_pred = pd.DataFrame({'rf' : dt.predict(X_test)})
    rg_pred = pd.DataFrame({'linreg' : dt.predict(X_test)})
    
    preds = pd.concat([dt_pred, rf_pred, rg_pred], axis = 1)
    
    y_pred = preds.apply(lambda row: row[int(random.uniform(0, 1) * 3 // 1)], axis = 1)

    return roc_auc_score(y_test, y_pred)

In [28]:
def combine(list_of_diseases, gender = None, new_features = False):
    dfs = [] 
    
    colnum = 202 if gender == 'M' else 152
    
    for fname in list_of_diseases:
        with open('data/' + fname) as f:
            content = f.readlines()
            
        content = [x.strip().split(' ')[:colnum] for x in content]
        cols = ['p_id', 'c_id'] + [fname + str(i) for i in range(colnum - 2)]
        
        df = pd.DataFrame(content, columns = cols)
        df = df.set_index('c_id')
        
        if new_features:
            # Using mean overweights other diseases but generally proportional
            df[fname + '_total'] = df.iloc[:, 1:].sum(axis = 1)
            df[fname + '_6mos'] = df.iloc[:, 1:28].sum(axis = 1)
            df[fname + '_1y'] = df.iloc[:, 1:53].sum(axis = 1)
            df[fname + '_2y'] = df.iloc[:, 1:105].sum(axis = 1)

        dfs.append(df)
        
    combined = pd.concat(dfs, sort = True)
    
    if new_features:
        filter_cols = [col for col in combined.columns if '_' not in col]
        combined['total'] = combined[filter_cols].sum(axis = 1)
    
    combined['sex'] = combined.p_id.apply(lambda x: x[0])
    combined['diagnosis'] = combined.p_id.apply(lambda x: x[1:4])
    combined['state'] = combined.p_id.apply(lambda x: x[4:6])
    combined['county'] = combined.p_id.apply(lambda x: x[6:9])

    combined, le = encode(combined)
    
    if gender:
        combined = combined[combined.sex == le.index(gender)]
        
    return combined